In [2]:
# 컨슈머 생성 (토픽으로 데이터를 전송)
from kafka import KafkaConsumer
import json
import time
from csv import reader

In [6]:
class MessageConsumer:
    def __init__(self,broker,topic,group_id):
        self.broker = broker
        self.topic = topic
        self.group_id = group_id
    def active_listener(self):
        params = {
            'bootstrap_servers' : self.broker,
            'group_id':self.group_id,
            'consumer_timeout_ms' : 1000*5,  # 5초
            # 토픽을 처음구독할때 또는 컨슈머의 오프셋이
            # 브로커에 의해 삭제될때 시작 오프셋을 설정
            'auto_offset_reset':'latest',
            'enable_auto_commit':False,
            # 역직렬화
            # ASCII로 디코딩 JSON형태로 파싱
            'value_deserializer':lambda x : json.loads(x.decode('ascii'))
        }
        consumer = KafkaConsumer(**params)
        consumer.subscribe(self.topic)  # 구독... 지정한 토픽의 데이터가 감지되면 읽는다(소비)
        start_time = time.time()
        # 데이터 출력 - 데이터를 읽었을때 처리할 로직
        try:
            with open(f'result_{self.topic}.txt','a'
                      , encoding='utf-8') as f:
                for data in consumer:
                    f.write(data.value)
            end_time = time.time()
            print(f'총 처리시간 : {end_time-start_time}' )
        except KeyboardInterrupt:
            print("KeyboardInterrupt")

In [7]:
#브로커와 토픽명을 지정
broker = 'localhost:9092'
topic = 'test'
group_id = 'consumer-1'

consumer = MessageConsumer(broker,topic,group_id)
consumer.active_listener()

총 처리시간 : 13.631734371185303
